In [0]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import uniform

In [9]:
data = pickle.load(open("/content/drive/My Drive/Anadi_Basak_with_vocab/data/vocab_train.pkl", "rb"))
X_pred = (pd.read_pickle("/content/drive/My Drive/Anadi_Basak_with_vocab/data/vocab_gs_test.pkl"))

gc.collect()

0

In [0]:
print(data.shape)
print(X_pred.shape)
data.loc[:,200].value_counts()

(613890, 201)
(8916, 200)


0    524186
1     89704
Name: 200, dtype: int64

In [0]:
1 - 89704/(89704+524186)

0.8538761015817166

In [10]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

target = data.loc[:,200]      
data.drop(columns = [200], axis=1, inplace=True)     # taking the training data features
                                 
# del data

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, random_state = 32) # in this our main data is split into train and test

del data, target
gc.collect()

0

In [11]:
y_train.value_counts()

0    366593
1     63130
Name: 200, dtype: int64

In [0]:
for i in range(2):
  X_train = X_train.append(X_train[y_train == 1])
  y_train = y_train.append(y_train[y_train == 1])

In [13]:
y_train.value_counts()

0    366593
1    252520
Name: 200, dtype: int64

# Model Fitting

In [14]:
from sklearn.model_selection import RandomizedSearchCV
import time

model = LogisticRegression(max_iter=10000000)

# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter distribution using uniform distribution
C = uniform(loc=0, scale=4)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

# Create randomized search 5-fold cross validation and 100 iterations
clf = RandomizedSearchCV(model, hyperparameters, random_state=42, n_iter=10, cv=3, verbose=1, n_jobs=-1, scoring = 'f1')

s = time.time()
best_model = clf.fit(X_train, y_train)
e = time.time()

# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])
print("Model Operation Completed in :",(e-s)/60,"min")

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  2.3min finished


Best Penalty: l2
Best C: 2.832290311184182
Model Operation Completed in : 2.540563174088796 min


In [15]:
model = LogisticRegression(max_iter=10000000, penalty = best_model.best_estimator_.get_params()['penalty'], C = best_model.best_estimator_.get_params()['C'])

import time

s = time.time()
model.fit(X_train, y_train)
e = time.time()

print("Model trained in", (e-s)/60,"min")
print(model.score(X_test, y_test))

pred = model.predict(X_test)
print(f1_score(pred, y_test))

Model trained in 0.23267778158187866 min
0.7626990720378787
0.3883329367800809


In [16]:
cutoff = np.arange(0.005, 0.995, 0.005)
f1 = []

pred = (model.predict_proba(X_test))[:,1]

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure(data = go.Scatter(x = cutoff, y = f1))
fig.show()

In [0]:
max_cutoff = cutoff[f1 == max(f1)]
max_cutoff

array([0.195])

In [0]:
print(accuracy_score(pred >= max_cutoff, y_test))

0.7644366254540715
